In [1]:
require 'cudnn'
require 'inn'
require 'image'
require 'dcnn'
Plot = require 'itorch.Plot'

-- Loads the mapping from net outputs to human readable labels
function load_synset()
  local file = io.open 'synset_words.txt'
  local list = {}
  while true do
    local line = file:read()
    if not line then break end
    table.insert(list, string.sub(line,11))
  end
  return list
end

-- Converts an image from RGB to BGR format and subtracts mean
function preprocess(im, img_mean)
  -- rescale the image
  local im3 = image.scale(im,224,224,'bilinear')*255
  -- RGB2BGR
  local im4 = im3:clone()
  im4[{1,{},{}}] = im3[{3,{},{}}]
  im4[{3,{},{}}] = im3[{1,{},{}}]

  -- subtract imagenet mean
  return im4 - image.scale(img_mean, 224, 224, 'bilinear')
end

-- Lua implementation of PHP scandir function
function scandir(directory)
    local i, t, popen = 0, {}, io.popen
    for filename in popen('ls -a "'..directory..'"'):lines() do
        if filename:match "%.png$" then
            i = i + 1
            t[i] = filename

        end
    end
    return t, i
end


fileDir = '/usr/local/data/imagenet/train'
fileList, total_files = scandir( fileDir )




-- Setting up networks 
print '==> Loading network'

--net = torch.load('/home/yusheng/Workspace/DeepLearning/models/zeilerNet/zeilerNet.net')
net = torch.load('/usr/local/data/zeilerNet/zeilerNet.net')
--net:add(nn.Sigmoid())
net:cuda()
-- as we want to classify, let's disable dropouts by enabling evaluation mode
net:evaluate()


print '==> Loading synsets'
synset_words = load_synset()

print '==> Loading image and imagenet mean'
--image_name = 'Goldfish3.jpg'
image_name = 'test_n01443537/n01443537_10025.JPEG'
--image_name = 'lena.jpg'
--image_name='people2.jpg'
img_mean_name = 'ilsvrc_2012_mean.t7'

im = image.load(image_name)
img_mean = torch.load(img_mean_name).img_mean:transpose(3,1)




-- Have to resize and convert from RGB to BGR and subtract mean
print '==> Preprocessing'
I = preprocess(im, img_mean)

sorted_prob, classes = net:forward(I:cuda()):view(-1):float():sort(true)

for i=1,5 do
  print('predicted class '..tostring(i)..': ', synset_words[classes[i]] .. ' ' .. sorted_prob[i] )
end

true_label = classes[1]

print(sorted_prob[1])

==> Loading network	


==> Loading synsets	
==> Loading image and imagenet mean	


==> Preprocessing	


predicted class 1: 	goldfish, Carassius auratus 12.869455337524	
predicted class 2: 	starfish, sea star 8.6937255859375	
predicted class 3: 	fountain 7.7210969924927	
predicted class 4: 	coral reef 6.7187376022339	
predicted class 5: 	conch 6.537576675415	
12.869455337524	


In [68]:
-- function NormalizedMax( fc_sorted_output ) 
--     local output_size = fc_sorted_output:size(1)
--     local normalized_output = {}
--     local sum = 0
    
--     for i=1, output_size  do
--         normalized_output[i] = fc_sorted_output[i]-fc_sorted_output[output_size]
--         sum = sum + normalized_output[i]
--     end
    
--     return normalized_output[1], sum/output_size
-- end

-- local average = 0
-- output_org, average = NormalizedMax( sorted_prob )
-- local ratio_of_mean_error = (output_org - average)/average 


-- print(output_org)
-- print(average)
-- print(ratio_of_mean_error)

In [2]:
-- local nThreads = 4
classifier = nn.Sequential()
classifier:add(net:get(2))
classifier:add(net:get(3))
--classifier:add(net:get(4))

local n=256

classification_curve = torch.Tensor(256)

--for n=1, 256 do
    topN_index = {}
    for i=1, n do
        topN_index[i] = { idx=0, diff=0 }
    end

    for i=1, net:get(1):get(13).output:size(1) do
        local incomplete_fm = net:get(1):get(13).output:clone()
        incomplete_fm[i]:zero()
        sorted_prob, classes = classifier:forward(incomplete_fm):view(-1):float():sort(true)
        --local diff = output_org - NormalizedMax( sorted_prob )
        local diff = output_org - sorted_prob[1]


        if topN_index[n].diff == 0 then 
            topN_index[n].diff = diff
            topN_index[n].idx = i

            local idx = n-1
            while idx>=1 and topN_index[idx].diff == 0 do
                topN_index[idx].diff = topN_index[idx+1].diff
                topN_index[idx].idx = topN_index[idx+1].idx
                topN_index[idx+1].diff = 0
                idx = idx - 1
            end
        
            if n==256 then
                for j=1,n-1 do 
                    for k=n,j+1,-1 do
                        -- print(k ..' '.. j )
                        if topN_index[k].diff > topN_index[k-1].diff then  
                            temp_diff = topN_index[k].diff
                            temp_idx = topN_index[k].idx
                            topN_index[k].diff = topN_index[k-1].diff
                            topN_index[k].idx = topN_index[k-1].idx
                            topN_index[k-1].diff = temp_diff
                            topN_index[k-1].idx = temp_idx
                        end
                    end
                end
            end
        else
            local temp_diff = 0
            local temp_idx = 0
            if n>1 then
                for j=1,n-1 do 
                    for k=n,j+1,-1 do
                        -- print(k ..' '.. j )
                        if topN_index[k].diff > topN_index[k-1].diff then  
                            temp_diff = topN_index[k].diff
                            temp_idx = topN_index[k].idx
                            topN_index[k].diff = topN_index[k-1].diff
                            topN_index[k].idx = topN_index[k-1].idx
                            topN_index[k-1].diff = temp_diff
                            topN_index[k-1].idx = temp_idx
                        end
                    end
                end
            end


    --         for j=1,5 do
    --             print( i .. '   ' .. top5_index[j].diff )
    --         end

            if topN_index[n].diff < diff then
                topN_index[n].diff = diff
                topN_index[n].idx = i
            end
        end

        --for i=1,5 do
        --print('predicted class '..tostring(i)..': ', synset_words[classes[1]] .. ' ' .. sorted_prob[1] )
        --end
    end


    for i=1,n do
        print( 'Top'.. i ..' drop down : ' .. topN_index[i].idx .. ' Diff: ' .. topN_index[i].diff) 
    end
    --print(incomplete_fm[1])
    --itorch.image(incomplete_fm)
    


Top1 drop down : 153 Diff: 19.858781337738	
Top2 drop down : 163 Diff: 19.416495800018	
Top3 drop down : 123 Diff: 18.731080532074	
Top4 drop down : 179 Diff: 18.041469097137	
Top5 drop down : 145 Diff: 17.634040355682	
Top6 drop down : 57 Diff: 17.406122684479	
Top7 drop down : 246 Diff: 17.32702589035	
Top8 drop down : 12 Diff: 17.323877811432	
Top9 drop down : 150 Diff: 17.288241863251	
Top10 drop down : 76 Diff: 17.2854180336	
Top11 drop down : 58 Diff: 17.180014133453	
Top12 drop down : 102 Diff: 17.109209537506	
Top13 drop down : 230 Diff: 16.999086856842	
Top14 drop down : 169 Diff: 16.840843677521	
Top15 drop down : 251 Diff: 16.730041980743	
Top16 drop down : 239 Diff: 16.718574047089	
Top17 drop down : 23 Diff: 16.667534351349	
Top18 drop down : 158 Diff: 16.637814044952	
Top19 drop down : 212 Diff: 16.610141277313	
Top20 drop down : 74 Diff: 16.592440128326	
Top21 drop down : 157 Diff: 16.575189113617	
Top22 drop down : 52 Diff: 16.569312572479	
Top23 drop down : 13 Diff: 16

 68 Diff: 16.287835597992	
Top94 drop down : 216 Diff: 16.284603595734	
Top95 drop down : 141 Diff: 16.283419132233	
Top96 drop down : 162 Diff: 16.282165050507	
Top97 drop down : 116 Diff: 16.281927585602	
Top98 drop down : 46 Diff: 16.281404972076	
Top99 drop down : 256 Diff: 16.278857707977	
Top100 drop down : 240 Diff: 16.277448177338	
Top101 drop down : 20 Diff: 16.274040699005	
Top102 drop down : 125 Diff: 16.27378988266	
Top103 drop down : 126 Diff: 16.27342748642	
Top104 drop down : 199 Diff: 16.272780895233	
Top105 drop down : 237 Diff: 16.27217912674	
Top106 drop down : 24 Diff: 16.271320819855	
Top107 drop down : 75 Diff: 16.271318912506	
Top108 drop down : 170 Diff: 16.270783901215	
Top109 drop down : 149 Diff: 16.270688533783	
Top110 drop down : 41 Diff: 16.269401073456	
Top111 drop down : 194 Diff: 16.268976688385	
Top112 drop down : 204 Diff: 16.266342639923	
Top113 drop down : 44 Diff: 16.26578950882	
Top114 drop down : 63 Diff: 16.26578950882	
Top115 drop down : 72 Dif

578502655	
Top184 drop down : 220 Diff: 16.196053028107	
Top185 drop down : 213 Diff: 16.194817066193	
Top186 drop down : 49 Diff: 16.193043231964	
Top187 drop down : 202 Diff: 16.192616939545	
Top188 drop down : 48 Diff: 16.192152500153	
Top189 drop down : 98 Diff: 16.19159078598	
Top190 drop down : 226 Diff: 16.190206050873	
Top191 drop down : 62 Diff: 16.188823223114	
Top192 drop down : 117 Diff: 16.184793949127	
Top193 drop down : 195 Diff: 16.1837849617	
Top194 drop down : 228 Diff: 16.183672428131	
Top195 drop down : 15 Diff: 16.182323932648	
Top196 drop down : 3 Diff: 16.182267665863	
Top197 drop down : 25 Diff: 16.17990064621	
Top198 drop down : 99 Diff: 16.178276538849	
Top199 drop down : 2 Diff: 16.177879810333	
Top200 drop down : 108 Diff: 16.177558422089	
Top201 drop down : 77 Diff: 16.158381938934	
Top202 drop down : 137 Diff: 16.158064365387	
Top203 drop down : 155 Diff: 16.15768289566	
Top204 drop down : 33 Diff: 16.157503604889	
Top205 drop down : 120 Diff: 16.155519962

In [3]:
for top_n=1, 256 do
    local incomplete_fm = net:get(1):get(13).output:clone()
    for i=1,net:get(1):get(13).output:size(1) do
        local drop = true
        for j=1,top_n do
            if i == topN_index[j].idx then
                drop = false
            end  
        end

        if drop == true then
            --print(i)
            incomplete_fm[i]:zero()
        end
    end

    sorted_prob, classes = classifier:forward(incomplete_fm):view(-1):float():sort(true)
    --output_org, average = NormalizedMax( sorted_prob )
    --local ratio_of_mean_error = (output_org - average)/average

    print('Top '..tostring(top_n)..' feature map classification: ', synset_words[classes[1]] .. ' ' .. sorted_prob[1] )
    
    --print(classes[1])
    if true_label == classes[1] then
        classification_curve[top_n]= sorted_prob[1] 
    else
        classification_curve[top_n] = 0
    end
end

Top 1 feature map classification: 	orange 2.9375867843628	


Top 2 feature map classification: 	goldfish, Carassius auratus 4.4019346237183	


Top 3 feature map classification: 	goldfish, Carassius auratus 6.3040170669556	


Top 4 feature map classification: 	goldfish, Carassius auratus 7.9453406333923	


Top 5 feature map classification: 	goldfish, Carassius auratus 9.5324649810791	


...

Top 252 feature map classification: 	goldfish, Carassius auratus 15.327188491821	


Top 253 feature map classification: 	goldfish, Carassius auratus 15.114090919495	


Top 254 feature map classification: 	goldfish, Carassius auratus 14.70991897583	


Top 255 feature map classification: 	goldfish, Carassius auratus 13.907990455627	


Top 256 feature map classification: 	goldfish, Carassius auratus 12.869455337524	


In [4]:
--print(classification_curve)
point_index = torch.Tensor(classification_curve:size(1))  
for i=1, classification_curve:size(1) do
    point_index[i] = i
end
plot = Plot():line( point_index, classification_curve,'red','Classification Curve'):legend(true):title('Line Plot Demo'):draw()


--gnuplot.plot( classification_curve )

In [5]:
for top_n=1, 256 do
    local incomplete_fm = net:get(1):get(13).output:clone()
    for i=1,net:get(1):get(13).output:size(1) do
        local drop = false
        for j=1,top_n do
            if i == topN_index[j].idx then
                drop = true
            end  
        end

        if drop == true then
            --print(i)
            print('Drop neuron :'.. i)
            incomplete_fm[i]:zero()
        end
    end

    sorted_prob, classes = classifier:forward(incomplete_fm):view(-1):float():sort(true)
    --output_org, average = NormalizedMax( sorted_prob )
    --local ratio_of_mean_error = (output_org - average)/average

    print('Top '..tostring(top_n)..' feature map classification: ', synset_words[classes[1]] .. ' ' .. sorted_prob[1] )
    
    --print(classes[1])
    if true_label == classes[1] then
        classification_curve[top_n]= sorted_prob[1] 
    else
        classification_curve[top_n] = 0
    end
end

Drop neuron :153	


Top 1 feature map classification: 	goldfish, Carassius auratus 9.276463508606	
Drop neuron :153	
Drop neuron :163	


Top 2 feature map classification: 	fountain 6.4844250679016	


Drop neuron :123	


Drop neuron :153	
Drop neuron :163	


...

Drop neuron :170	
Drop neuron :171	
Drop neuron :172	
Drop neuron :173	
Drop neuron :174	
Drop neuron :175	
Drop neuron :176	
Drop neuron :177	
Drop neuron :178	
Drop neuron :179	
Drop neuron :180	
Drop neuron :181	
Drop neuron :182	
Drop neuron :183	
Drop neuron :184	
Drop neuron :185	
Drop neuron :186	
Drop neuron :187	
Drop neuron :188	
Drop neuron :189	
Drop neuron :190	
Drop neuron :191	
Drop neuron :192	
Drop neuron :193	
Drop neuron :194	
Drop neuron :195	
Drop neuron :196	
Drop neuron :197	
Drop neuron :198	
Drop neuron :199	
Drop neuron :200	
Drop neuron :201	
Drop neuron :202	
Drop neuron :203	
Drop neuron :204	
Drop neuron :205	
Drop neuron :206	
Drop neuron :207	
Drop neuron :208	
Drop neuron :209	
Drop neuron :210	
Drop neuron :211	
Drop neuron :212	
Drop neuron :213	
Drop neuron :214	
Drop neuron :215	
Drop neuron :216	
Drop neuron :217	
Drop neuron :218	
Drop neuron :219	
Drop neuron :220	
Drop neuron :221	
Drop neuron :222	
Drop neuron :223	
Drop neuron :224	
Drop neuro

Top 255 feature map classification: 	broccoli 2.5427675247192	
Drop neuron :1	
Drop neuron :2	
Drop neuron :3	
Drop neuron :4	
Drop neuron :5	
Drop neuron :6	
Drop neuron :7	
Drop neuron :8	
Drop neuron :9	
Drop neuron :10	
Drop neuron :11	
Drop neuron :12	
Drop neuron :13	
Drop neuron :14	
Drop neuron :15	
Drop neuron :16	
Drop neuron :17	
Drop neuron :18	
Drop neuron :19	
Drop neuron :20	
Drop neuron :21	
Drop neuron :22	
Drop neuron :23	
Drop neuron :24	
Drop neuron :25	
Drop neuron :26	
Drop neuron :27	
Drop neuron :28	
Drop neuron :29	
Drop neuron :30	
Drop neuron :31	
Drop neuron :32	
Drop neuron :33	
Drop neuron :34	
Drop neuron :35	
Drop neuron :36	
Drop neuron :37	
Drop neuron :38	
Drop neuron :39	
Drop neuron :40	
Drop neuron :41	
Drop neuron :42	
Drop neuron :43	
Drop neuron :44	
Drop neuron :45	
Drop neuron :46	
Drop neuron :47	
Drop neuron :48	
Drop neuron :49	
Drop neuron :50	
Drop neuron :51	
Drop neuron :52	
Drop neuron :53	
Drop neuron :54	
Drop neuron :55	
Drop neuron

Drop neuron :70	
Drop neuron :71	
Drop neuron :72	
Drop neuron :73	
Drop neuron :74	
Drop neuron :75	
Drop neuron :76	
Drop neuron :77	
Drop neuron :78	
Drop neuron :79	
Drop neuron :80	
Drop neuron :81	
Drop neuron :82	
Drop neuron :83	
Drop neuron :84	
Drop neuron :85	
Drop neuron :86	
Drop neuron :87	
Drop neuron :88	
Drop neuron :89	
Drop neuron :90	
Drop neuron :91	
Drop neuron :92	
Drop neuron :93	
Drop neuron :94	
Drop neuron :95	
Drop neuron :96	
Drop neuron :97	
Drop neuron :98	
Drop neuron :99	
Drop neuron :100	
Drop neuron :101	
Drop neuron :102	
Drop neuron :103	
Drop neuron :104	
Drop neuron :105	
Drop neuron :106	
Drop neuron :107	
Drop neuron :108	
Drop neuron :109	
Drop neuron :110	
Drop neuron :111	
Drop neuron :112	
Drop neuron :113	
Drop neuron :114	
Drop neuron :115	
Drop neuron :116	
Drop neuron :117	
Drop neuron :118	
Drop neuron :119	
Drop neuron :120	
Drop neuron :121	
Drop neuron :122	
Drop neuron :123	
Drop neuron :124	
Drop neuron :125	
Drop neuron :126	
Drop

Drop neuron :173	
Drop neuron :174	
Drop neuron :175	
Drop neuron :176	
Drop neuron :177	
Drop neuron :178	
Drop neuron :179	
Drop neuron :180	
Drop neuron :181	
Drop neuron :182	
Drop neuron :183	
Drop neuron :184	
Drop neuron :185	
Drop neuron :186	
Drop neuron :187	
Drop neuron :188	
Drop neuron :189	
Drop neuron :190	
Drop neuron :191	
Drop neuron :192	
Drop neuron :193	
Drop neuron :194	
Drop neuron :195	
Drop neuron :196	
Drop neuron :197	
Drop neuron :198	
Drop neuron :199	
Drop neuron :200	
Drop neuron :201	
Drop neuron :202	
Drop neuron :203	
Drop neuron :204	
Drop neuron :205	
Drop neuron :206	
Drop neuron :207	
Drop neuron :208	
Drop neuron :209	
Drop neuron :210	
Drop neuron :211	
Drop neuron :212	
Drop neuron :213	
Drop neuron :214	
Drop neuron :215	
Drop neuron :216	
Drop neuron :217	
Drop neuron :218	
Drop neuron :219	
Drop neuron :220	
Drop neuron :221	
Drop neuron :222	
Drop neuron :223	
Drop neuron :224	
Drop neuron :225	
Drop neuron :226	
Drop neuron :227	
Drop neuro

Top 256 feature map classification: 	shoe shop, shoe-shop, shoe store 0.024763148277998	


In [6]:
--print(classification_curve)
point_index = torch.Tensor(classification_curve:size(1))  
for i=1, classification_curve:size(1) do
    point_index[i] = i
end
plot = Plot():line( point_index, classification_curve,'red','Classification Curve'):legend(true):title('Line Plot Demo'):draw()


In [186]:
net

nn.Sequential {
  [input -> (1) -> (2) -> (3) -> output]
  (1): nn.Sequential {
    [input -> (1) -> (2) -> (3) -> (4) -> (5) -> (6) -> (7) -> (8) -> (9) -> (10) -> (11) -> (12) -> (13) -> (14) -> output]
    (1): cudnn.SpatialConvolution(3 -> 96, 7x7, 2,2, 1,1)
    (2): cudnn.ReLU
    (3): inn.SpatialSameResponseNormalization
    (4): cudnn.SpatialMaxPooling
    (5): cudnn.SpatialConvolution(96 -> 256, 5x5, 2,2)
    (6): cudnn.ReLU
    (7): inn.SpatialSameResponseNormalization
    (8): cudnn.SpatialMaxPooling
    (9): cudnn.SpatialConvolution(256 -> 384, 3x3, 1,1, 1,1)
    (10): cudnn.ReLU
    (11): cudnn.SpatialConvolution(384 -> 384, 3x3, 1,1, 1,1)
    (12): cudnn.ReLU
    (13): cudnn.SpatialConvolution(384 -> 256, 3x3, 1,1, 1,1)
    (14): cudnn.ReLU
  }
  (2): inn.SpatialPyramidPooling
  (3): nn.Sequential {
    [input -> (1) -> (2) -> (3) -> (4) -> (5) -> (6) -> (7) -> output]
    (1): nn.Linear(12800 -> 4096)
    (2): cudnn.ReLU
    (3): nn.Dropout(0.500000)
    (4): nn.Linear(40

              {
                  padW : 1
                  nInputPlane : 3
                  output : CudaTensor - size: 96x110x110
                  gradInput : CudaTensor - size: 3x224x224
                  iDesc : cdata<struct cudnnTensorStruct *[1]>: 0x40eb55d8
                  iSize : LongStorage - size: 4
                  convDesc : cdata<struct cudnnConvolutionStruct *[1]>: 0x41ef75a0
                  output_offset : 1161600
                  gradBias : CudaTensor - size: 96
                  algType : cdata<enum 761 [?]>: 0x4194c7d0
                  extraBuffer : CudaTensor - size: 147
                  extraBufferSizeInBytes : 588
                  oDescForBias : cdata<struct cudnnTensorStruct *[1]>: 0x416dc7c0
                  oDesc : cdata<struct cudnnTensorStruct *[1]>: 0x4089ff50
                  groups : 1
                  dH : 2
                  dW : 2
                  nOutputPlane : 96
 

                 bias : CudaTensor - size: 96
                  kH : 7
                  weight_offset : 14112
                  finput : CudaTensor - empty
                  biasDesc : cdata<struct cudnnTensorStruct *[1]>: 0x4030b1c0
                  input_offset : 150528
                  weight : CudaTensor - size: 96x3x7x7
                  train : false
                  gradWeight : CudaTensor - size: 96x3x7x7
                  fgradInput : CudaTensor - empty
                  kW : 7
                  padH : 1
                  weightDesc : cdata<struct cudnnFilterStruct *[1]>: 0x416b4a18
                }
              2 : 
                cudnn.ReLU
                {
                  inplace : true
                  mode : CUDNN_ACTIVATION_RELU
                  train : false
   

               output : CudaTensor - size: 96x110x110
                  gradInput : CudaTensor - empty
                  nElem : 1161600
                  iDesc : cdata<struct cudnnTensorStruct *[1]>: 0x407380a8
                }
              3 : 
                inn.SpatialSameResponseNormalization
                {
                  alpha : 5e-05
                  train : false
                  output : CudaTensor - size: 96x110x110
                  gradInput : CudaTensor - empty
                  size : 3
                  beta : 0.75
                  modules : nn.Sequential {
  [input -> (1) -> (2) -> output]
  (1): nn.ConcatTable {
    input
      |`-> (1): nn.Identity
      |`-> (2): nn.Sequential {
      |      [input -> (1) -> (2) -> (3) -> (4) -> (5) -> (6) -> output]
      |      (1): nn.SpatialZeroPadding(l=1,r=1,t=1,b=1)
      |      (2): nn.Power
      |      (3): nn.SpatialAveragePooling(3,3,1,1)
      |      (4): nn.MulConstant
      |      (5): nn.AddConstant
      

            dH : 2
                  dW : 2
                  padH : 0
                  gradInput : CudaTensor - size: 96x110x110
                  oDesc : cdata<struct cudnnTensorStruct *[1]>: 0x408a1dd0
                  iSize : LongStorage - size: 4
                  iDesc : cdata<struct cudnnTensorStruct *[1]>: 0x40ea5468
                  poolDesc : cdata<struct cudnnPoolingStruct *[1]>: 0x40616c78
                  mode : CUDNN_POOLING_MAX
                  train : false
                  kW : 3
                  ceil_mode : true
                  padW : 0
                  output : CudaTensor - size: 96x55x55
                  kH : 3
                }
              5 : 
                cudnn.SpatialConvolution(96 -> 256, 5x5, 2,2)
                {
                  padW : 0
            

...

   4 : 
                {
                  1 : 6
                  2 : 6
                }
            }
          train : false
          output : CudaTensor - size: 12800
          gradInput : CudaTensor - empty
          module : 
            nn.Concat {
              input
                |`-> (1): nn.Sequential {
                |      [input -> (1) -> (2) -> output]
                |      (1): nn.SpatialAdaptiveMaxPooling
                |      (2): nn.View
                |    }
                |`-> (2): nn.Sequential {
                |      [input -> (1) -> (2) -> output]
                |      (1): nn.SpatialAdaptiveMaxPooling
                |      (2): nn.View
                |    }
                |`-> (3): nn.Sequential {
                |      [input -> (1) -> (2) -> output]
                |      (1): nn.SpatialAdaptiveMaxPooling
                |      (2): nn.View
                |    }
                |`-> (4): nn.Sequential {
                |      [input -> (1) -> 

        nn.Sequential {
          [input -> (1) -> (2) -> (3) -> (4) -> (5) -> (6) -> (7) -> output]
          (1): nn.Linear(12800 -> 4096)
          (2): cudnn.ReLU
          (3): nn.Dropout(0.500000)
          (4): nn.Linear(4096 -> 4096)
          (5): cudnn.ReLU
          (6): nn.Dropout(0.500000)
          (7): nn.Linear(4096 -> 1000)
        }
        {
          gradInput : CudaTensor - empty
          modules : 
            {
              1 : 
                nn.Linear(12800 -> 4096)
                {
                  gradBias : CudaTensor - size: 4096
                  weight : CudaTensor - size: 4096x12800
                  train : false
                  bias : CudaTensor - size: 4096
                  gradInput : CudaTensor - empty
                  output : CudaTensor - size: 4096
                  gradWeight : CudaTensor - size: 4096x12800
                }
              2 : 
                cudnn.ReLU
                {
                  inplace : true
                

   nElem : 4096
                  iDesc : cdata<struct cudnnTensorStruct *[1]>: 0x40b338f8
                }
              3 : 
                nn.Dropout(0.500000)
                {
                  v2 : false
                  noise : CudaTensor - empty
                  train : false
                  p : 0.5
                  gradInput : CudaTensor - empty
                  output : CudaTensor - size: 4096
                }
              4 : 
                nn.Linear(4096 -> 4096)
                {
                  gradBias : CudaTensor - size: 4096
                  weight : CudaTensor - size: 4096x4096
                  train : false
                  bias : CudaTensor - size: 4096
                  gradInput : CudaTensor - empty
                  output : CudaTensor - size: 4096
                  gradWeight : CudaTensor - size: 4096x4096
          

      }
              5 : 
                cudnn.ReLU
                {
                  inplace : true
                  mode : CUDNN_ACTIVATION_RELU
                  train : false
                  output : CudaTensor - size: 4096
                  gradInput : CudaTensor - empty
                  nElem : 4096
                  iDesc : cdata<struct cudnnTensorStruct *[1]>: 0x419545b0
                }
              6 : 
                nn.Dropout(0.500000)
                {
                  v2 : false
                  noise : CudaTensor - empty
                  train : false
                  p : 0.5
                  gradInput : CudaTensor - empty
                  output : CudaTensor - size: 4096
                }
              7 : 
                nn.Linear(4096 -> 1000)
                {


                  gradBias : CudaTensor - size: 1000
                  weight : CudaTensor - size: 1000x4096
                  train : false
                  bias : CudaTensor - size: 1000
                  gradInput : CudaTensor - empty
                  output : CudaTensor - size: 1000
                  gradWeight : CudaTensor - size: 1000x4096
                }
            }
          train : false
          output : CudaTensor - size: 1000
        }
    }
  train : false
  output : CudaTensor - size: 1000
}


In [22]:
x1 = torch.randn(40)
y1 = torch.Tensor(40)
for i=1,40 do
    y1[i] = i 
end
print(x1)
plot = Plot():line(y1, x1, 'red','example'):legend(true):title('Line Plot Demo'):draw()

 0.9400
 0.1357
 1.1384
-0.9214
 0.6813
 0.2097
-1.0063
 0.4904
 0.7521
 0.1191
-0.7687
 0.1466
-1.1040
 1.4171
 1.2943
-0.4709
-0.3478
-0.2723
-1.7604
-0.8848
-1.3617
 0.9320
-0.5779
-0.7112
-0.8270
-0.4356
 0.1186
-0.4663
-0.8055
-0.9754
-1.2868
-0.5876
 0.2873
 0.4034
-0.1160
 1.3642
 0.0904
 1.4338
 2.6844
-1.0716
[torch.DoubleTensor of size 40]

